# EXPERIENCES ALEATOIRES CONTROLEES

Nous avons maintenant suffisamment de connaissances du logiciel pour étudier un exemple d'expérience aléatoire contrôlée. 

L'exercice suivant se base sur l'article de Pascaline Dupas and Jonathan Robinson : 

"Savings Constraints and Microenterprise Development: Evidence
from a Field Experiment in Kenya". *American Economic Journal: Applied Economics*, 5(1):163-192, 2013"

Les auteurs souhaitent savoir si l'accès limité à la possibilité d'épargner pour les travailleurs ruraux indépendants empêche le développement de leur petite entreprise (donc in fine la croissance économique et le développement).

Pour vérifier cela, ils ont recours à une expérience contrôlée dans le village de Bumala au Kenya. Ils s'intéressent à deux types de microentrepreneurs : les taxi-bicyclettes (buda), et les vendeurs de rue. 

### 1. Données

Dans une **première étape**, les enquêteurs sur place à Bumala dressent une liste la plus exhaustive possible des vendeurs de rue et des taxi-bicyclettes en les accostant.

Ouvrir la base de données: **TD5_database.dta** et l'inspecter grâce à un résumé rapide de statistiques descriptives.

In [ ]:
library(foreign)
d<-read.dta("TD5_database.dta")
summary(d)

*Occupation* est la variable qui donne l'occupation des personnes : =1 si buda (homme taxi-bicyclette), =2 si femme vendeuse, =3 si homme vendeur. Transformer la variable en catégorie: "Buda", "Vendeur Femme", "Vendeur Homme".

In [ ]:
d$occupation<-factor(d$occupation, labels = c("Buda","Vendeur Femme","Vendeur Homme"))
levels(d$occupation)

Combien de vendeuses, vendeurs et taxis ont été recensés au début de l'expérience et intégrés à notre échantillon initial (variable *occupation*) ? Quelle est leur proportion respective dans l'échantillon ? 

In [ ]:
table(d$occupation)
prop.table(table(d$occupation))

Ces individus sont ensuite affectés de manière aléatoire entre groupe de traitement et groupe de contrôle suivant une stratification par genre et occupation (homme/femme, et vendeur/taxi-bicyclette). 

a) Définir la stratification. Pourquoi stratifier l'échantillon par genre et occupation ? 

b) Combien a-t-on d'hommes taxi-bicyclette dans le groupe de traitement/contrôle (variables *occupation* et *treatment* ? Quelle est la proportion des hommes taxi dans le groupe de traitement ? Note: traitement = 1, contrôle = 0.

In [ ]:
#b) Combien a-t-on d'hommes taxi-bicyclette dans le groupe de traitement/contrôle (variables *occupation* et *treatment* ?
table(d$occupation,d$treatment) 
#proportion de chque groupe dans l'échantillon: attention ce n'est pas la réponse à la question mais ça montre bien la stratification.
prop.table(table(d$occupation,d$treatment))
#proportion des hommes taxi dans le groupe de traitement: c'est la réponse : 8%.
prop.table(table(d$occupation,d$treatment),margin=2)

### 2. Design de l'expérience 
Au groupe de traitement est donnée la possibilité d'ouvrir un compte épargne gratuitement (ce qui est normalement payant, et plutôt cher). Au groupe de contrôle n'est pas offerte cette possibilité. Ensuite, les individus des deux groupes reportent chaque jour pendant trois mois des informations détaillées sur leur revenu, leurs dépenses, les transferts monétaires, leurs heures de travail dans un livret.

Selon vous, quels sont les différents problèmes posés par le design de cette expérience ?

En général, dans quel cas l'attrition pose-t-elle un problème ? Donner un exemple ici.

Calculer le taux d'attrition pour les femmes et les hommes, puis pour les femmes et les
hommes par groupe de traitement/contrôle. On utilise pour cela la variable *bg_gender* qui vaut 1 si la personne est un homme, 0 sinon. On utilise aussi la variable *inlogs* qui vaut 1 si la personne a rempli son livret jusqu'au bout, 0 sinon. Commenter.

In [ ]:
#Optionnel : on recode ces variables en facteurs pour avoir un beau tableau. 
d$bg_gender<-factor(d$bg_gender, labels=c("Female","Male"))
d$inlogs<-factor(d$inlogs, labels=c("Non rempli","Rempli"))
#Tableau
table(d$inlogs,d$bg_gender,d$treatment) 
prop.table(table(d[d$treatment==0,]$inlogs,d[d$treatment==0,]$bg_gender), margin=2)
prop.table(table(d[d$treatment==1,]$inlogs,d[d$treatment==1,]$bg_gender), margin=2)

Comment vérifier que la procédure d'assignation aléatoire (randomization) n'est pas
remise en cause par l'attrition ? Conduire ce test pour la sous-population des hommes
qui ont rempli leur livret, puis des femmes. 

On devrait appliquer ce test à l'ensemble des variables de baseline : *bg_age*, *bg_married*, *bg_num_children*, *bg_educ*, *bg_rosca*, *bg_rosca_contrib_lyr*. **Mais pour être plus bref on fera le test sur deux variables : *bg_age*, et *bg_educ***. Conclure.

In [ ]:
#test pour les femmes sans attrition: on teste H0 égalité des moyennes entre groupes C/T (contre H1)
#sous échantillon : d[d$bg_gender == "Female" & d$inlogs == "Rempli",]
a<-t.test(bg_age ~ treatment, data=d[d$bg_gender == "Female" & d$inlogs == "Rempli",])
b<-t.test(bg_educ ~ treatment, data=d[d$bg_gender == "Female" & d$inlogs == "Rempli",])
print(c(a$statistic,b$statistic))
print(c(a$p.value,b$p.value))
#malgré l'attrition, les groupes sont toujours équilibrés (pas de différence de moyenne)

#idem pour les hommes sans attrition
#sous échantillon : d[d$bg_gender == "Male" & d$inlogs == "Rempli",]
a<-t.test(bg_age ~ treatment, data=d[d$bg_gender == "Male" & d$inlogs == "Rempli",])
b<-t.test(bg_educ ~ treatment, data=d[d$bg_gender == "Male" & d$inlogs == "Rempli",])
print(c(a$statistic,b$statistic))
print(c(a$p.value,b$p.value))
#l'attrition n'a pas été aléatoire pour les hommes car le niveau d'éducation n'est plus le même en moyenne entre C et T

On dispose de la variable d'ouverture du compte : *opened_acct* = 1 si le compte épargne a été ouvert par l'individu, 0 sinon. L'adhésion au traitement (ouverture effective d'un compte-épargne suite à assignation
au traitement) est-elle parfaite ?

In [ ]:
d$opened_acct<-factor(d$opened_acct, labels=c("Non ouvert","Ouvert"))
table(d$opened_acct,d$treatment)
prop.table(table(d$opened_acct,d$treatment),margin=2)

On aurait souhaité imposer que tous les traités adoptent le compte épargne, tandis
qu'aucun des individus du groupe de contrôle n'y ait accès. Quel est le problème que
poserait une telle expérience ?

### 3. Analyse des résultats

On s'intéresse maintenant à l'effet du programme de gratuité du compte
épargne sur le niveau d'épargne, et au delà sur le niveau d'investissement dans la microentreprise.

Soient :
- $Y_i$ la variable dépendante (niveau d'épargne : *bank_savings*, niveau d'investissement : *investment_t5*, nombre d'heures travaillées : *total_hours*, revenus de l'entreprise : *revenues_t5*) ;
- $D_i$ une variable qui vaut 1 si l'individu a été assigné au groupe de traitement : *treatment*,
- $X_i$ la matrice de l'ensemble des variables de contrôle : statut marital *bg_married*, nombre d'enfants *bg_num_children*, âge *bg_age*, alphabétisation *bg_kis_read*, et enfin les variables de stratification : hommes vendeurs *bg_malevendor*, taxi-bicyclettes
*bg_boda*.

On se retreint au sous-échantillon des personnes qui ont rempli lelivret jusqu'à la fin (*inlogs*="Rempli"). On estime d'abord un modèle très simple par MCO : $Y_{i} = \alpha +  \gamma D_{i} + \epsilon_{i}$

a) Mesure-t-on le TOT ou l'ITT ?

b) Estimer avec comme variable dépendante l'épargne : *bank_savings*. Interpréter.

c) Calculer le TOT. Est-ce logique qu'il soit plus élevé que l'ITT ?

In [ ]:
#b) 
#attention on doit d'abord remplacer d par son sous-échantillon sans attrition !
d <- subset(d,d$inlogs == "Rempli") 
# écrire ensuite la régression : on l'appelle lm1
#lm1<-

In [ ]:
#c)TOT=ITT/pi1-pi0
coef(lm1)
ITT <- coef(lm1)[2]

pi<-data.frame(prop.table(table(d$opened_acct,d$treatment),margin=2))
pi

# calculer le TOT à partir de lm1 et de pi: 
#TOT<-

On estime ensuite l'ITT avec un modèle linéaire multiple: $Y_{i} = \alpha + X_{i} \beta + \gamma D_{i} + \epsilon_{i}$. 

a) Pourquoi ajouter des variables de contrôle à la régression simple?

b) Estimer (cf. ci-dessus pour définition de $X_i$) et interpréter. 

In [ ]:
#lm2<-
#lm3<-
#lm4<-
#lm5<-

<font color=green> 
    En moyenne, le groupe qui s'est vu offrir le compte épargne épargne près de 10 shillings de plus que l'autre par jour. Il investit 74 shillings de plus et gagne 62 shillings de plus par jour. L'épargne n'a pas d'impact sur le nombre d'heures travaillées.
    </font>

c) Ajouter "femmes vendeurs" ($bg\_femalevendor$, soit la troisième catégorie de stratification avec hommes vendeurs et taxi-bicyclette) dans la matrice des variables de contrôle $X_{i}$. Que se passe-t-il? Pourquoi?

In [ ]:
lm6<-lm(bank_savings~treatment+bg_married+bg_age+bg_educ+bg_num_children+bg_kis_read+bg_malevendor+bg_boda+bg_femalevendor, data=d)
summary(lm6)

Représentez côte à côte dans un graphique l'effet causal de la possibilité d'ouvrir un compte sur l'épargne, selon le modèle simple et le modèle multiple. 

In [ ]:
#on utilise lm1 pour extraire coef et intervalle:
est1<-
est1_sd<-
c1_inf<-
c1_sup<-
#idem lm2:
est2<-
est2_sd<-
c2_inf<-
c2_sup<-

In [ ]:
#création du data frame 
est <- 

In [ ]:
#graphique
library(ggplot2)
ggplot(...)

### 4. Réflexion

Il s'avère que ce compte épargne n'est pas comme un compte standard : il ne rapporte pas d'intérêt, et il comporte même un coût de dépôt et de retrait ; pour ainsi dire, le taux d'intérêt
réel est négatif ( !)

Pourquoi utiliser ce compte dans ces conditions ? Au delà de l'étude de la causalité, l'économiste doit chercher à expliquer le mécanisme qui amène les individus à bénéficier d'un compte épargne même à taux d'intéret négatif.

a) Selon vous, quelles peuvent être les motivations des individus pour épargner sur ce compte au lieu de réinvestir directement dans leur entreprise (ce qui leur éviterait le taux négatif) ?

b) Selon vous, quelles peuvent être les motivations des individus pour épargner sur ce compte au lieu d'épargner chez soi ?